In [1]:
#!pip install beautifulsoup4

In [1]:
import pandas as pd
import numpy as np
import requests
import lxml
from bs4 import BeautifulSoup
import urllib.request 
import sys
import time
from datetime import datetime

In [2]:
df_player_scores = pd.read_csv('../data/player_scores.csv')

In [3]:
df_player_scores

,player_name,season,week,score
0,Marco Wölfli,100,0,3
1,Marco Wölfli,100,1,1
2,Marco Wölfli,100,2,2
3,Marco Wölfli,100,3,2
4,Marco Wölfli,100,4,4
...,...,...,...,...
7454,Guillaume Hoarau,2100,9,0
7455,Guillaume Hoarau,2100,10,0
7456,Guillaume Hoarau,2100,11,0
7457,Guillaume Hoarau,2100,12,4


In [39]:
def html_call(url):
    # here we define the headers for the request
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}

    # this request object will integrate your URL and the headers defined above
    req = urllib.request.Request(url=url, headers=headers)
    #req = urllib.request.Request(url=url, headers=headers)
    # calling urlopen this way will automatically handle closing the request
    
    time.sleep(0.5)
    with urllib.request.urlopen(req,timeout=20) as response:
        page_html = response.read()
    return page_html

In [9]:
def parse_play_date(play_date, season):    
    # skip EM (3, 13) and WM (8, 18)
    season_to_year = {
        0 : 2011,
        1 : 2012,
        3 : 2012,
        4 : 2013,
        5 : 2013,
        6 : 2014,
        8 : 2014,
        9 : 2015,
        10 : 2015,
        11 : 2016,
        13 : 2016,
        14 : 2017,
        15 : 2017,
        16 : 2018,
        18 : 2018,
        19 : 2019,
        20 : 2019}
    datetime_object = datetime.strptime(f'{play_date[4:]} {season_to_year[season]}','%d.%m. %H:%M %Y')
    return datetime_object  

In [61]:
def super_league_parse():
    scores_per_week = {}
    season_list = []
    week_list = []
    date_times_list = []
    #for season in range(21):
    yb_matches_list = []
    #for season in [2019]:
    for season in range(2011,2020):
        # skip EM (3, 13) and WM (8, 18)
        #if season in(2,12,7,17):
            #continue
        try:
            url = f'https://www.weltfussball.com/alle_spiele/sui-super-league-{season}-{season+1}/'
            # url = f'https://www.mydreamteam.ch/spielplan_super_league.php?season={season_str}'
            print(url)
            html = html_call(url)
            soup = BeautifulSoup(html,'html.parser')
            matches = soup.find_all('table',{'class':'standard_tabelle'})
            df_matches = pd.read_html(str(matches[0]),header=0)[0]
            df_matches.columns = ['date','time','home','a','guest','b','c','d']
            df_matches = df_matches.ffill()
            df_matches_yb = df_matches[(df_matches['home'] == 'BSC Young Boys') | (df_matches['guest'] == 'BSC Young Boys')]
            yb_matches_list.append(df_matches_yb)            
        except:
            print(sys.exc_info())
            return 0
        
    return yb_matches_list

In [62]:
yb_matches_list = super_league_parse()

https://www.weltfussball.com/alle_spiele/sui-super-league-2011-2012/
https://www.weltfussball.com/alle_spiele/sui-super-league-2012-2013/
https://www.weltfussball.com/alle_spiele/sui-super-league-2013-2014/
https://www.weltfussball.com/alle_spiele/sui-super-league-2014-2015/
https://www.weltfussball.com/alle_spiele/sui-super-league-2015-2016/
https://www.weltfussball.com/alle_spiele/sui-super-league-2016-2017/
https://www.weltfussball.com/alle_spiele/sui-super-league-2017-2018/
https://www.weltfussball.com/alle_spiele/sui-super-league-2018-2019/
https://www.weltfussball.com/alle_spiele/sui-super-league-2019-2020/


In [86]:
def date_to_dream_season(date):
    dto = datetime.strptime(date,'%d.%m.%Y')

    # skip EM (3, 13) and WM (8, 18)
    if dto > datetime.strptime('01.07.2011','%d.%m.%Y') and dto < datetime.strptime('01.01.2012','%d.%m.%Y'):
        season = 100
    elif dto > datetime.strptime('01.01.2012','%d.%m.%Y') and dto < datetime.strptime('01.07.2012','%d.%m.%Y'):
        season = 200
    elif dto > datetime.strptime('01.07.2012','%d.%m.%Y') and dto < datetime.strptime('01.01.2013','%d.%m.%Y'):
        season = 400
    elif dto > datetime.strptime('01.01.2013','%d.%m.%Y') and dto < datetime.strptime('01.07.2013','%d.%m.%Y'):
        season = 500
    elif dto > datetime.strptime('01.07.2013','%d.%m.%Y') and dto < datetime.strptime('01.01.2014','%d.%m.%Y'):
        season = 600
    elif dto > datetime.strptime('01.01.2014','%d.%m.%Y') and dto < datetime.strptime('01.07.2014','%d.%m.%Y'):
        season = 700
    elif dto > datetime.strptime('01.07.2014','%d.%m.%Y') and dto < datetime.strptime('01.01.2015','%d.%m.%Y'):
        season = 900
    elif dto > datetime.strptime('01.01.2015','%d.%m.%Y') and dto < datetime.strptime('01.07.2015','%d.%m.%Y'):
        season = 1000
    elif dto > datetime.strptime('01.07.2015','%d.%m.%Y') and dto < datetime.strptime('01.01.2016','%d.%m.%Y'):
        season = 1100
    elif dto > datetime.strptime('01.01.2015','%d.%m.%Y') and dto < datetime.strptime('01.07.2016','%d.%m.%Y'):
        season = 1200
    elif dto > datetime.strptime('01.07.2016','%d.%m.%Y') and dto < datetime.strptime('01.01.2017','%d.%m.%Y'):
        season = 1400
    elif dto > datetime.strptime('01.01.2016','%d.%m.%Y') and dto < datetime.strptime('01.07.2017','%d.%m.%Y'):
        season = 1500
    elif dto > datetime.strptime('01.07.2017','%d.%m.%Y') and dto < datetime.strptime('01.01.2018','%d.%m.%Y'):
        season = 1600
    elif dto > datetime.strptime('01.01.2017','%d.%m.%Y') and dto < datetime.strptime('01.07.2018','%d.%m.%Y'):
        season = 1700
    elif dto > datetime.strptime('01.07.2018','%d.%m.%Y') and dto < datetime.strptime('01.01.2019','%d.%m.%Y'):
        season = 1900
    elif dto > datetime.strptime('01.01.2018','%d.%m.%Y') and dto < datetime.strptime('01.07.2019','%d.%m.%Y'):
        season = 2000
    elif dto > datetime.strptime('01.07.2019','%d.%m.%Y') and dto < datetime.strptime('01.01.2020','%d.%m.%Y'):
        season = 2100
    else:
        season = 2200
    return season

In [100]:
playday_list = list(range(0,18)) * 2
for season_matches in yb_matches_list:
    season_matches['play_day'] = playday_list
    season_matches['season'] = season_matches['date'].apply(lambda x:date_to_dream_season(x) )

In [101]:
df_yb_matches_enriched = pd.concat(yb_matches_list)

In [103]:
df_player_scores_parsed = pd.merge(df_player_scores, df_yb_matches_enriched,  how='left', left_on=['season','week'], right_on = ['season','play_day'])

In [12]:
df_player_scores_parsed.to_csv( '../data/player_scores_parsed.csv', index=False)

In [109]:
df_player_scores_parsed[['player_name','score','date']].to_csv( '../data/player_scores_parsed.csv', index=False)